Trip log joins are an operational necessity. However, they do not process fast enough. Let's get them running more quickly.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.transit import gtfs_realtime_pb2
import sys; sys.path.append("../src/")
from processing import parse_feeds_into_trip_logbook, merge_trip_logbooks

with open("../src/tests/data/gtfs_realtime_pull_1.dat", "rb") as f:
    gtfs_r0 = gtfs_realtime_pb2.FeedMessage()
    gtfs_r0.ParseFromString(f.read())
with open("../src/tests/data/gtfs_realtime_pull_2.dat", "rb") as f:
    gtfs_r1 = gtfs_realtime_pb2.FeedMessage()
    gtfs_r1.ParseFromString(f.read())

left_logbook = parse_feeds_into_trip_logbook([gtfs_r0], [0])
right_logbook = parse_feeds_into_trip_logbook([gtfs_r1], [1])

In [ ]:
# Slow!
result = merge_trip_logbooks([left_logbook, right_logbook])

> /home/alex/Desktop/mta-data-exploration/src/processing.py(558)merge_trip_logbooks()
-> left = dict()
(Pdb) c


../src/processing.py:636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  entries['action'] = 'STOPPED_OR_SKIPPED'
../src/processing.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  entries['maximum_time'] = next_entries['maximum_time']


The above makes it obvious that the problem is that we are running `_join_trip_logs`, a multi-second operation, 162 times here. So we need to raise that functions' execution speed.

In [3]:
from processing import _join_trip_logs

Old code:

In [5]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
_join_trip_logs(left_logbook['055450_4..N06R'], right_logbook['055450_4..N06R'])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

../src/processing.py:631: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  #     entry['action'] = 'STOPPED_OR_SKIPPED'
../src/processing.py:632: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  #     entry['maximum_time'] = next_entry['maximum_time']


3.096 _join_trip_logs  processing.py:591
└─ 3.059 __setitem__  pandas/core/series.py:716
   └─ 3.059 _check_is_chained_assignment_possible  pandas/core/generic.py:1512
      └─ 3.059 _check_setitem_copy  pandas/core/generic.py:1533



New code (using `numpy` indexed assignment):

In [9]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
_join_trip_logs(left_logbook['055450_4..N06R'], right_logbook['055450_4..N06R'])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

0.152 _join_trip_logs  processing.py:591
├─ 0.113 __setitem__  pandas/core/indexing.py:135
│  └─ 0.112 _setitem_with_indexer  pandas/core/indexing.py:233
│     └─ 0.112 setter  pandas/core/indexing.py:455
│        └─ 0.112 __setitem__  pandas/core/frame.py:2405
│           └─ 0.112 _set_item  pandas/core/frame.py:2473
│              └─ 0.110 _check_setitem_copy  pandas/core/generic.py:1533
├─ 0.014 <listcomp>  processing.py:622
│  ├─ 0.009 __init__  pandas/core/frame.py:252
│  │  └─ 0.008 _init_dict  pandas/core/frame.py:349
│  │     └─ 0.006 _arrays_to_mgr  pandas/core/frame.py:5391
│  │        └─ 0.005 create_block_manager_from_arrays  pandas/core/internals.py:4259
│  │           └─ 0.004 form_blocks  pandas/core/internals.py:4270
│  │              └─ 0.002 __new__  pandas/indexes/base.py:142
│  │                 └─ 0.002 _handle_fromlist  <frozen importlib._bootstrap>:2264
│  └─ 0.004 transpose  pandas/core/frame.py:1835
│     └─ 0.004 transpose  pandas/core/generic.py:471
│        

/home/alex/miniconda3/envs/mta-data-exploration/lib/python3.4/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


This is a 20x speedup. However, the full merge still takes 20 seconds (!), due to `__setitem__` calls still in `_join_trip_logs`. It's not transparent to me where these are coming from, so I spent some time muddling about in it.

In [15]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
result = merge_trip_logbooks([left_logbook, right_logbook])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

6.024 _join_trip_logs  processing.py:590
├─ 2.242 <listcomp>  processing.py:621
│  ├─ 1.421 __init__  pandas/core/frame.py:252
│  │  └─ 1.350 _init_dict  pandas/core/frame.py:349
│  │     ├─ 0.969 _arrays_to_mgr  pandas/core/frame.py:5391
│  │     │  └─ 0.906 create_block_manager_from_arrays  pandas/core/internals.py:4259
│  │     │     ├─ 0.701 form_blocks  pandas/core/internals.py:4270
│  │     │     │  ├─ 0.235 equals  pandas/indexes/base.py:1643
│  │     │     │  │  └─ 0.193 array_equivalent  pandas/types/missing.py:245
│  │     │     │  │     └─ 0.065 array_equal  numpy/core/numeric.py:2476
│  │     │     │  ├─ 0.189 _simple_blockify  pandas/core/internals.py:4385
│  │     │     │  │  └─ 0.134 make_block  pandas/core/internals.py:2649
│  │     │     │  │     └─ 0.112 __init__  pandas/core/internals.py:1811
│  │     │     │  │        └─ 0.103 __init__  pandas/core/internals.py:96
│  │     │     │  │           └─ 0.075 mgr_locs  pandas/core/internals.py:216
│  │     │     │  └─ 0.13

We got another 33% speedup by avoiding `DataFrame` transforms.

In [22]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
result = merge_trip_logbooks([left_logbook, right_logbook])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

3.782 _join_trip_logs  processing.py:590
├─ 1.591 __getitem__  pandas/core/frame.py:2035
│  └─ 1.463 _getitem_array  pandas/core/frame.py:2078
│     ├─ 1.308 take  pandas/core/generic.py:1650
│     │  ├─ 1.110 take  pandas/core/internals.py:3943
│     │  │  ├─ 0.792 reindex_indexer  pandas/core/internals.py:3813
│     │  │  │  ├─ 0.493 <listcomp>  pandas/core/internals.py:3848
│     │  │  │  │  ├─ 0.255 take_nd  pandas/core/internals.py:2126
│     │  │  │  │  │  └─ 0.210 take_nd  pandas/core/categorical.py:1481
│     │  │  │  │  │     └─ 0.168 take_nd  pandas/core/algorithms.py:1010
│     │  │  │  │  │        └─ 0.079 _maybe_promote  pandas/types/cast.py:227
│     │  │  │  │  └─ 0.225 take_nd  pandas/core/internals.py:1001
│     │  │  │  │     └─ 0.162 take_nd  pandas/core/algorithms.py:1010
│     │  │  │  │        └─ 0.053 _maybe_promote  pandas/types/cast.py:227
│     │  │  │  └─ 0.280 __init__  pandas/core/internals.py:2745
│     │  │  │     ├─ 0.172 _rebuild_blknos_and_blklocs  pan

WIP:

In [29]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
result = merge_trip_logbooks([left_logbook, right_logbook])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

1.664 _join_trip_logs  processing.py:590
├─ 0.745 __getitem__  pandas/core/indexing.py:1302
│  └─ 0.731 _getitem_axis  pandas/core/indexing.py:1599
│     ├─ 0.664 _get_loc  pandas/core/indexing.py:104
│     │  └─ 0.656 _ixs  pandas/core/frame.py:1953
│     │     ├─ 0.293 __init__  pandas/core/series.py:135
│     │     │  ├─ 0.146 _sanitize_array  pandas/core/series.py:2817
│     │     │  │  └─ 0.094 _try_cast  pandas/core/series.py:2834
│     │     │  │     ├─ 0.047 _possibly_cast_to_datetime  pandas/types/cast.py:765
│     │     │  │     └─ 0.033 is_extension_type  pandas/types/common.py:301
│     │     │  ├─ 0.057 __init__  pandas/core/internals.py:4031
│     │     │  │  └─ 0.048 make_block  pandas/core/internals.py:2649
│     │     │  │     └─ 0.018 __init__  pandas/core/internals.py:1811
│     │     │  ├─ 0.029 _validate_dtype  pandas/core/generic.py:131
│     │     │  │  └─ 0.026 _coerce_to_dtype  pandas/types/common.py:320
│     │     │  └─ 0.026 __setattr__  pandas/core/generic.

In [23]:
_join_trip_logs(left_logbook['055450_4..N06R'], right_logbook['055450_4..N06R'])

> /home/alex/Desktop/mta-data-exploration/src/processing.py(619)_join_trip_logs()
-> left.loc[:, 'stop_id'] = pd.Categorical(left['stop_id'], stations)
(Pdb) stations
['250N', '239N', '235N', '234N', '423N', '420N', '419N', '418N', '640N', '635N', '631N', '629N', '626N', '621N', '416N', '415N', '414N', '413N', '412N', '411N', '410N', '409N', '408N', '407N', '406N', '405N', '402N', '401N']
(Pdb) len(stations)
28
(Pdb) len(set(stations))
28
(Pdb) c


,index,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time
0,0,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,NaN,250N,0
1,1,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,239N,0
2,2,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,235N,0
3,3,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,234N,0
4,4,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,423N,0
5,5,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,420N,0
6,6,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,419N,0
7,7,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,418N,0
8,8,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,640N,0
9,9,055450_4..N06R,4,STOPPED_OR_SKIPPED,0,nan,635N,0


Begin TODO: debug this.

In [1]:
import os

logs = [f for f in os.listdir("./data/subway_time_20160512") if f != 'arch.tar.xz' 
        and 'si' not in f and 'l' not in f]

In [2]:
logs[:5]

['gtfs-20160512T0415Z',
 'gtfs-20160512T1759Z',
 'gtfs-20160512T2155Z',
 'gtfs-20160512T0610Z',
 'gtfs-20160513T0153Z']

In [3]:
from google.transit import gtfs_realtime_pb2

In [4]:
def parse_feed(filepath):
    with open(filepath, "rb") as f:
        try:
            fm = gtfs_realtime_pb2.FeedMessage()
            fm.ParseFromString(f.read())
            return fm
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            return None

Note: the data above comes from the next notebook. You can get it yourself by using the following magical incantation:

    pip install requests gtfs-realtime-bindings
    python -c "import requests; r = requests.get('http://data.mytransit.nyc.s3.amazonaws.com/subway_time/2016/2016-05/subway_time_20160512.tar.xz'); open('arch.tar.xz', 'wb').write(r.content)"
    tar xvfJ arch.tar.xz
    python -c "from google.transit import gtfs_realtime_pb2; test_example = gtfs_realtime_pb2.FeedMessage().ParseFromString(open('gtfs-20160512T0400Z', 'rb').read()); print(type(test_example))"

In [7]:
from tqdm import tqdm

In [12]:
feeds = [parse_feed("./data/subway_time_20160512/" + l) for l in tqdm(logs[:6])]


100%|██████████| 6/6 [00:01<00:00,  5.24it/s]


In [16]:
import sys; sys.path.append("../src/")
from processing import parse_feeds_into_trip_logbook

In [19]:
# logbooks = [
#     parse_feeds_into_trip_logbook(feeds[0:3], [0, 1, 2]), 
#     parse_feeds_into_trip_logbook(feeds[3:6], [3, 4, 5])
# ]

AssertionError: 

End TODO: debug this.